In [1]:
from pathlib import Path

In [2]:
lines = Path("day22.txt").read_text().splitlines()
steps = [(tok[0] == 'on', [tuple([int(r) for r in t[2:].split("..")]) for t in tok[1].split(",")])
         for line in lines for tok in [line.split()]]

## Part 1

In [3]:
cubes = set()
for on, ((x0, x1), (y0, y1), (z0, z1)) in steps:
    for x in range(max(x0, -50), min(x1, 50) + 1):
        for y in range(max(y0, -50), min(y1, 50) + 1):
            for z in range(max(z0, -50), min(z1, 50) + 1):
                if on:
                    cubes.add((x, y, z))
                elif (x, y, z) in cubes:
                    cubes.remove((x, y, z))

len(cubes)

648681

## Part 2

In [5]:
def prod(x):
    p = 1
    for x in x: p *= x
    return p

def sizes(c):
    return [max(b - a + 1, 0) for a, b in c]

def size(c):
    return prod(sizes(c))

def intersect(c0, c1):
    return [(max(a[0], b[0]), min(a[1], b[1])) for a, b in zip(c0, c1)]

def difference(c0, c1):
    # optimize the difference for an empty intersection
    if size(i := intersect(c0, c1)) == 0:
        return [c0]
    
    (ix0, ix1), (iy0, iy1), (iz0, iz1) = i
    (cx0, cx1), (cy0, cy1), (cz0, cz1) = c0
    
    # create candidate difference cuboids
    # . a pair along the x-axis on either side of the subtrahend
    # . a pair along the y-axis, except for the cells in the first pair
    # . finally a pair along the z-axis, to cap off the difference
    cs = [
        ((cx0, ix0 - 1), (cy0, cy1), (cz0, cz1)),
        ((ix1 + 1, cx1), (cy0, cy1), (cz0, cz1)),
        ((ix0, ix1), (cy0, iy0 - 1), (cz0, cz1)),
        ((ix0, ix1), (iy1 + 1, cy1), (cz0, cz1)),
        ((ix0, ix1), (iy0, iy1), (cz0, iz0 - 1)),
        ((ix0, ix1), (iy0, iy1), (iz1 + 1, cz1)),
    ]
    
    # ignore any empty/invalid cuboids
    return [c for c in cs if size(c) > 0]

cubes = []
for on, c in steps:
    # for either on/off steps, remove the indicated cuboids from all existing
    cubes = sum((difference(o, c) for o in cubes), start=[])
    
    # if turning on, we now know that the new cuboid has no
    # intersection with any existing, so simply add it
    if on:
        cubes.append(c)

sum(size(c) for c in cubes)

1302784472088899